In [1]:
data_source = ['math-amc', 'math-aime', 'gpqa']

In [1]:
import os

os.environ['HF_HOME'] = '/nas-ssd2/joykirat/.cache/huggingface'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")
# model = LLM(model="Qwen/Qwen3-4B", gpu_memory_utilization=0.95, tensor_parallel_size=1, task="embed")





INFO 08-13 22:37:38 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-13 22:37:39 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model.encode("Hello, world! </think>")

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.65it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


[PoolingRequestOutput(request_id='0', outputs=PoolingOutput(data=tensor([ 0.0008,  0.0162, -0.0035,  ..., -0.0022, -0.0210,  0.0208])), prompt_token_ids=[9707, 11, 1879, 0, 220, 151668], finished=True)]

In [5]:
import json
data = []
with open('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_linear_reward_with_summary_attention/rollout/val_140.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
        

In [6]:
import numpy as np

accuracy_by_source = {}
for line in data:
    if line['data_source'] not in accuracy_by_source:
        accuracy_by_source[line['data_source']] = []
    accuracy_by_source[line['data_source']].append(line['final_reward'])
    
for source in accuracy_by_source:
    mean_accuracy = np.mean(accuracy_by_source[source])
    print(source, mean_accuracy, len(accuracy_by_source[source]))
    
    

math-amc 0.7927710843373494 415
gpqa 0.44771573604060916 985
math-aime 0.45 440


In [7]:
import numpy as np
response_length_list = {}
from tqdm import tqdm
for line in tqdm(data):
    if line['data_source'] not in response_length_list:
        response_length_list[line['data_source']] = []
    response_length_list[line['data_source']].append(len(tokenizer.encode(line['response'])))
    
for source in response_length_list:
    mean_response_length = np.mean(response_length_list[source])
    print(source, mean_response_length, len(response_length_list[source]))
    
    
    
            
            
            

100%|██████████| 1840/1840 [00:11<00:00, 162.69it/s]

math-amc 4156.530120481928 415
gpqa 4193.119796954315 985
math-aime 6651.779545454546 440


In [60]:
line['response']

"<think>\nOkay, so I need to figure out for how many integers n between 1 and 22 inclusive, the denominator k_n of the harmonic series H_n = 1 + 1/2 + 1/3 + ... + 1/n is less than L_n, the least common multiple of 1 through n. \n\nFirst, I remember that when you add fractions, the denominator of the sum is the least common multiple of the denominators divided by some factors. But since the harmonic series is being expressed in reduced form, the denominator k_n is L_n divided by some factors that come from the numerator. So, k_n = L_n / d, where d is the greatest common divisor of the numerator and L_n. Therefore, k_n < L_n is equivalent to d > 1. So, the question becomes: for how many n between 1 and 22 is the numerator of H_n (when expressed in lowest terms) not coprime with L_n?\n\nAlternatively, since H_n = h_n / k_n = (sum_{i=1}^n 1/i) = (some numerator) / L_n, but reduced. Therefore, k_n = L_n / gcd(numerator, L_n). Therefore, k_n < L_n iff gcd(numerator, L_n) > 1.\n\nTherefore, t

In [61]:
len(tokenizer.encode(line['response']))

10000